import time
plc=1
time.sleep(1500*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
for i in range (1,6):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=ECG5000(i)
os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,6):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print( i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
print('shape ecg=',np.shape(ecg))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (500, 141)
first_test_shape= (4500, 141)
classes_quantity= 5
tr_lbls=	 {1, 2, 3, 4, 5}
Count_labels= [292 177  10  19   2]
max(train_feature_Altitude)= 5.0
min(train_feature_Altitude)= -5.7976446
first_train_sample=
 [ 1.         -0.11252183 -2.8272038  -3.7738969  -4.3497511  -4.376041
 -3.4749863  -2.1814082  -1.8182865  -1.2505219  -0.47749208 -0.36380791
 -0.49195659 -0.42185509 -0.30920086 -0.4959387  -0.34211867 -0.35533627
 -0.36791303 -0.31650279 -0.41237405 -0.47167181 -0.41345783 -0.36461703
 -0.44929829 -0.47141866 -0.42477658 -0.46251673 -0.55247236 -0.47537519
 -0.6942     -0.7018681  -0.59381178 -0.66068415 -0.71383066 -0.76980688
 -0.67228161 -0.65367605 -0.63940562 -0.55930228 -0.59167032 -0.49322332
 -0.46305183 -0.30164382 -0.23273401 -0.12505488 -0.15394314 -0.0243574
 -0.06560876  0.03499926  0.06193522  0.07119542  0.12392505  0.10312371
  0.22522849  0.12868305  0.30248315  0.25727621  0.19635161  0.17938297
  0.24472863  0.34121687  0.32820441 

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [3]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,6):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,6):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 292	2 177	3 10	4 19	5 2	
 max = 292


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [4]:
beta=6.0                  #به توان رساندن سهم روش داده افزایی متناسب عملکردش
alpha=10                   #چند برابر کردن داده افزایی هر کلاس متناسب ضعیف بودن آن کلاس
X=1                       #داده افزایی تا چند برابر کلاس اکثریت
mx_wndws=int(X*mx_wndws)  #داده افزایی تا چند برابر کلاس اکثریت

scors0=np.array([98,93,41,42,11])                  #Class Scorses without Augmentation
AgScr=1+alpha*(100-scors0)/100            #Class Specific

f_scr=np.array([58,61,61,63,62,58,57,62]) #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
f_scr_difrnc=f_scr-f_scr[0]
f_scr_difrnc=f_scr_difrnc**beta
f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
aug_amnt=np.empty((0,len(f_scr_rtio)-1), int) 

for i in range (1,6):
    aug_amnt=np.append(aug_amnt,[f_scr_rtio[1:]*(mx_wndws*AgScr[i-1]-cls_wndws[i-1])],axis=0)
    #vlum_win=mx_wndws-cls_wndws[i-1]             #میزان داده افزایی در روش های عمیق (پنجره)
aug_amnt=np.int16(np.round(aug_amnt))
print(np.int16(100*f_scr_rtio))
print(aug_amnt)

[ 0  2  2 61 16  0  0 16]
[[   2    2   36    9    0    0    9]
 [   9    9  197   52    0    0   52]
 [  58   58 1239  325    0    0  325]
 [  57   57 1216  319    0    0  319]
 [  83   83 1786  468    0    0  468]]


In [5]:
xtrain=np.empty((0,smpl_rte+1), float) 

for i in range (1,6):                                     #تعریف آرایه ی پنجره های کلاس ها
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    if aug_amnt[i-1,0]>0:
        methd=10
        sort=-2  #-5=Dis_sam_Cls=qlty1 #-4=var_Dis=qlty2 #-3=DisSm/DisOthr=qlty3  #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5
        dp_slct_dstrb=1            #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
        #xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort),axis=0)
        aaa=np.array(srtd_data(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort))
    if aug_amnt[i-1,1]>0:
        methd=20
        sort=-3
        dp_slct_dstrb=1
        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort),axis=0)
    if aug_amnt[i-1,2]>0:
        methd=30
        sort=-2
        dp_slct_dstrb=2
        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort),axis=0)
    if aug_amnt[i-1,3]>0:
        methd=41
        sort=-5
        dp_slct_dstrb=1
        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort),axis=0)
    if aug_amnt[i-1,4]>0:
        methd=50
        sort=-2
        dp_slct_dstrb=1
        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort),axis=0)
    if aug_amnt[i-1,5]>0:
        methd=60
        sort=-1
        dp_slct_dstrb=1
        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort),axis=0)
    if aug_amnt[i-1,6]>0:
        methd=70
        sort=-5
        dp_slct_dstrb=1
        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort),axis=0)
    print('class ', i, 'train shape = ', np.shape(xtrain))

final shape=  (2, 487)
final shape=  (2, 487)
final shape=  (36, 487)
final shape=  (9, 487)
final shape=  (9, 487)
class  1 train shape =  (348, 481)
final shape=  (9, 487)
final shape=  (9, 487)
final shape=  (197, 487)
final shape=  (52, 487)
final shape=  (52, 487)
class  2 train shape =  (835, 481)
final shape=  (58, 487)
final shape=  (58, 487)
final shape=  (1239, 487)
final shape=  (325, 487)
final shape=  (325, 487)
class  3 train shape =  (2792, 481)
final shape=  (57, 487)
final shape=  (57, 487)
final shape=  (1216, 487)
final shape=  (319, 487)
final shape=  (319, 487)
class  4 train shape =  (4722, 481)
final shape=  (83, 487)
final shape=  (83, 487)
final shape=  (1751, 487)
final shape=  (468, 487)
final shape=  (468, 487)
class  5 train shape =  (7494, 481)


In [6]:
aug_amnt

array([[   2,    2,   36,    9,    0,    0,    9],
       [   9,    9,  197,   52,    0,    0,   52],
       [  58,   58, 1239,  325,    0,    0,  325],
       [  57,   57, 1216,  319,    0,    0,  319],
       [  83,   83, 1786,  468,    0,    0,  468]], dtype=int16)

In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[   0  348  487 1957 1930 2772]


In [8]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,6):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [9]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [10]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,6):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [11]:
#print(np.max(wndws_tst))
for i in range (1,6):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,6):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  5.398327935600255
min magnitude class 1  =  -8.366738588591014
after normalizing >>
max magnitude class 1  =  1.1992656245468574
min magnitude class 1  =  -1.3646635578155866

 cls 2  >> 
max magnitude class 2  =  7.557156182756754
min magnitude class 2  =  -6.623263697325152
after normalizing >>
max magnitude class 2  =  1.601376484880539
min magnitude class 2  =  -1.0399178586519304

 cls 3  >> 
max magnitude class 3  =  3.427621598045422
min magnitude class 3  =  -6.827926378639614
after normalizing >>
max magnitude class 3  =  0.8321950056113407
min magnitude class 3  =  -1.0780390417381505

 cls 4  >> 
max magnitude class 4  =  3.033042703743151
min magnitude class 4  =  -5.658319868425961
after normalizing >>
max magnitude class 4  =  0.7586993679354934
min magnitude class 4  =  -0.8601840638382838

 cls 5  >> 
max magnitude class 5  =  2.4648784169429363
min magnitude class 5  =  -4.7540569
after normalizing >>
max magnitude class 5  =  0.6

In [12]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -1.3646635578155866
max tst = 1.601376484880539


In [13]:
len(wndws_test)

4500

In [14]:
np.shape(wndws_test)

(4500, 481)

In [15]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [16]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [17]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [18]:
np.shape(wndws1)

(292, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [19]:
Xvalid=np.array(Xtest)
yvalid=np.array(ytest)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [20]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtest)
y_valid=np.array(ytest)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (7494, 480)
y_train =>  (7494,)
X_test  =>  (4500, 480)
y_test  =>  (4500,)
X_valid  =>  (4500, 480)
y_valid  =>  (4500,)


In [21]:
y_train=np.array(ytrain)
np.min(y_train)

1

In [22]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [23]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 50
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()
BatchNormalization()
model_crs.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
#model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


model_crs.summary()
print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
150/150 [==============================] - 5s 12ms/step - loss: 0.5272 - accuracy: 0.8113 - val_loss: 1.1095 - val_accuracy: 0.5440
Epoch 2/1000
150/150 [==============================] - 4s 24ms/step - loss: 0.1615 - accuracy: 0.9442 - val_loss: 0.7983 - val_accuracy: 0.6916
Epoch 3/1000
150/150 [==============================] - 2s 10ms/step - loss: 0.1125 - accuracy: 0.9604 - val_loss: 0.5917 - val_accuracy: 0.8424
Epoch 4/1000
150/150 [==============================] - 2s 10ms/step - loss: 0.0924 - accuracy: 0.9676 - val_loss: 0.5013 - val_accuracy: 0.8676
Epoch 5/1000
150/150 [==============================] - 2s 10ms/step - loss: 0.0727 - accuracy: 0.9764 - val_loss: 0.5670 - val_accuracy: 0.8873
Epoch 6/1000
150/150 [==============================] - 2s 10ms/step - loss: 0.0644 - accuracy: 0.9765 - val_loss: 0.7611 - val_accuracy: 0.8229
Epoch 7/1000
150/150 [==============================] - 2s 10ms/step - loss: 0.0562 - accuracy: 0.9821 - val_loss: 0.5005 - val_ac

 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 2000)              2050000   
                                                                 
 dense_1 (Dense)             (None, 6)                 12006     
                                                                 
Total params: 2,240,886
Trainable params: 2,240,886
Non-trainable params: 0
_________________________________________________________________
Accuracy=  0.9331111311912537


In [24]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [25]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
print(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

141/141 [==============================] - 0s 2ms/step
[[2598    9    4    3   13]
 [  17 1482   23   62    6]
 [  14   21   36    8    7]
 [   4   85    5   77    4]
 [   7    5    4    0    6]]
              precision    recall  f1-score   support

           1       0.98      0.99      0.99      2627
           2       0.93      0.93      0.93      1590
           3       0.50      0.42      0.46        86
           4       0.51      0.44      0.47       175
           5       0.17      0.27      0.21        22

    accuracy                           0.93      4500
   macro avg       0.62      0.61      0.61      4500
weighted avg       0.93      0.93      0.93      4500



In [26]:
print(np.bincount(y_train)[1:])

[ 348  487 1957 1930 2772]
